## Steps

So note that we have two different sampling methods, "UAR" and "POP", and the labels we have for our 7 tasks are split between them. Thus, we must train on two different image datasets at once.

Our multitask model will also have 7 heads, so we must edit our ResNet18 to accomplish that

We should also save the model parameters for future use.

## Creating our training and test dataset

In [1]:
import io as python_io
import time
import math
from tqdm import tqdm
import sklearn.metrics
import torch
from loguru import logger
from torchvision import transforms
from torch import nn
from torchvision import models
import numpy as np
from pathlib import Path
import pickle
from mosaiks import config as c
from mosaiks import transforms as m_transforms
from mosaiks.featurization import RemoteSensingSubgridDataset, chunks
from mosaiks.utils import io, spatial
from mosaiks.solve import data_parser as parse
import pandas as pd
from torchinfo import summary
from tensorboardX import SummaryWriter
from tqdm.contrib import tzip
import torch.nn.functional as F
import uuid
import skimage.io
import skimage.transform

env variable MOSAIKS_HOME not defined; setting to: "/home/ubuntu/cs230/mosaiks-paper"
If not desired, please reset os.environ["MOSAIKS_NAME"]


In [2]:
tasks_UAR = ["treecover", "elevation", "population",]
tasks_POP = ["nightlights", "income", "roads", "housing",]


tasks = [
    "treecover",
    "elevation",
    "population",
    "nightlights",
    "income",
    "roads",
    "housing",
]

data_home = Path(c.data_dir) / "raw" / "imagery"
data_home_UAR = data_home / "CONTUS_UAR"
data_home_POP = data_home / "CONTUS_POP"

In [3]:
def grab_labels(task):
    c_local = io.get_filepaths(c, task)
    c_app = getattr(c_local, task)
    Y = io.get_Y(c_local, c_app["colname"])
    lons, lats = spatial.ids_to_ll(
        Y.index,
        c.grid_dir,
        c_local.grid["area"],
        c_local.images["zoom_level"],
        c_local.images["n_pixels"],
    )
    latlons = np.vstack((np.array(lats), np.array(lons))).T.astype("float64")
    ids, Y, latlons = m_transforms.dropna_and_transform(
        Y.index.values, Y.values, latlons, c_app
    )
    return Y, latlons, ids

In [4]:
def split_train_test(ids, Y, ratio=0.8):
    seed = 0
    r = np.random.RandomState(seed=seed)
    
    n = ids.shape[0]
    
    test_n = round((1 - ratio) * n)
    train_n = n - test_n
    
    shuffled_idx = r.choice(n, n, replace=False)
    train_idx = shuffled_idx[:train_n]
    test_idx = shuffled_idx[train_n:]
    
    return ids[train_idx], Y[train_idx], ids[test_idx], Y[test_idx]

This code loads labels for each task based on sampling method.

In [5]:
dfs_UAR = pd.DataFrame()
for task in tasks_UAR:
    Y_task, ll_task, ids_task = grab_labels(task)

    Y_and_ids = np.vstack([Y_task, ids_task]).T
    
    df = pd.DataFrame(Y_and_ids, columns=[task, "id"])
    df = df.set_index("id")
    if dfs_UAR.empty:
        dfs_UAR = df
    else:
        dfs_UAR = dfs_UAR.merge(df, how='outer', on='id')
        
display(dfs_UAR)
    
dfs_POP = pd.DataFrame()
for task in tasks_POP:
    Y_task, ll_task, ids_task = grab_labels(task)

    Y_and_ids = np.vstack([Y_task, ids_task]).T
    
    df = pd.DataFrame(Y_and_ids, columns=[task, "id"])
    df = df.set_index("id")
    if dfs_POP.empty:
        dfs_POP = df
    else:
        dfs_POP = dfs_POP.merge(df, how='outer', on='id')
        
display(dfs_POP)

,treecover,elevation,population
id,,,
"1000,105",91.223158,1462.463364,NaN
"1000,1067",11.715897,1919.119465,2.234753
"1000,1080",0.0,1814.329174,0.385035
"1000,1219",0.162632,2079.739485,0.010085
"1000,122",89.316842,1544.912699,NaN
...,...,...,...
"999,76",96.171579,156.416732,NaN
"999,914",0.003158,1534.612079,1.980177
"999,942",0.0,1524.94773,NaN


,nightlights,income,roads,housing
id,,,,
"1000,114",0.0,53277.708883,0.0,NaN
"1000,1457",2.793769,85920.0,5856.211968,5.003458
"1000,1458",2.374133,85920.0,3708.122387,5.144845
"1000,1459",2.813775,62022.434843,5458.893879,5.091291
"1000,1463",1.613503,61151.985279,2711.385912,5.042022
...,...,...,...,...
"999,962",3.164506,74250.611331,6009.25,NaN
"999,963",2.678835,73374.52862,5856.974,NaN
"999,964",3.302048,73351.713585,8132.549,NaN


Some tasks don't have labels for all images, so we can detect that here.

In [6]:
(ids_train_UAR, Y_train_UAR, ids_test_UAR, Y_test_UAR) = split_train_test(dfs_UAR.index.to_numpy(), dfs_UAR.loc[:, dfs_UAR.columns != 'id'].to_numpy(dtype='float32'))
(ids_train_POP, Y_train_POP, ids_test_POP, Y_test_POP) = split_train_test(dfs_POP.index.to_numpy(), dfs_POP.loc[:, dfs_POP.columns != 'id'].to_numpy(dtype='float32'))

print(Y_train_UAR.shape)
print(Y_train_POP.shape)
print(Y_train_UAR.dtype)
print(Y_train_POP.dtype)

for i in range(len(tasks_POP)):
    print(f"NaN count in train, task {tasks_POP[i]}: {np.sum(np.isnan(Y_train_POP[:,i]))}/80000")
    print(f"NaN count in test, task {tasks_POP[i]}: {np.sum(np.isnan(Y_test_POP[:,i]))}/20000")

(80000, 3)
(80000, 4)
float32
float32
NaN count in train, task nightlights: 1/80000
NaN count in test, task nightlights: 0/20000
NaN count in train, task income: 6927/80000
NaN count in test, task income: 1696/20000
NaN count in train, task roads: 0/80000
NaN count in test, task roads: 0/20000
NaN count in train, task housing: 38064/80000
NaN count in test, task housing: 9581/20000


In [7]:
def transform_img_inputs(augment):
    out = [transforms.ToPILImage(), transforms.CenterCrop(224)]
    if augment:
        out += [transforms.RandomResizedCrop(224), transforms.RandomHorizontalFlip(), transforms.RandomRotation(20)]
    out += [transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]
    return transforms.Compose(out)

In [8]:
def get_dataloader(data_home, Y, ids, batch_size=16, shuffle=True, num_workers=4, augment=False):
    transform = transform_img_inputs(augment)
    r_grid = RemoteSensingSubgridDataset(data_home, Y, ids, transform=transform)
    return torch.utils.data.DataLoader(r_grid, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

## Models

This model is our simple model with a linear head. Note that we have two heads for easier training on UAR tasks and POP tasks, should we choose to train them simultaneously.

In [9]:
class MultiTaskModel(nn.Module):
    def __init__(self):
        super(MultiTaskModel, self).__init__()
        #shared part
        self.resnet18 = models.resnet18(pretrained=False)
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()
        
        self.sampling = nn.ModuleList()
        self.sampling.add_module('UAR', nn.Linear(num_ftrs, len(tasks_UAR)))
        self.sampling.add_module('POP', nn.Linear(num_ftrs, len(tasks_POP)))

    def forward(self, X, sampling):
        # shared part
        resnet_output = self.resnet18(X)

        # sampling specific parts
        if sampling == 'UAR':
            return self.sampling.UAR(resnet_output)
        elif sampling == 'POP':
            return self.sampling.POP(resnet_output)

This model adds another linear layer with a ReLU before the final outputs. Otherwise, it is the same.

In [10]:
class MultiTaskModelTwo(nn.Module):
    def __init__(self):
        super(MultiTaskModelTwo, self).__init__()
        #shared part
        self.resnet18 = models.resnet18(pretrained=False)
        num_ftrs = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()
        num_hidden = int(num_ftrs / 2)
        
        self.sampling = nn.ModuleList()
        self.sampling.add_module('UAR', nn.Sequential(nn.Linear(num_ftrs, num_hidden), nn.ReLU(), nn.Linear(num_hidden, len(tasks_UAR))))
        self.sampling.add_module('POP', nn.Sequential(nn.Linear(num_ftrs, num_hidden), nn.ReLU(), nn.Linear(num_hidden, len(tasks_POP))))

    def forward(self, X, sampling):
        # shared part
        resnet_output = self.resnet18(X)

        # sampling specific parts
        if sampling == 'UAR':
            return self.sampling.UAR(resnet_output)
        elif sampling == 'POP':
            return self.sampling.POP(resnet_output)

This is a wrapper that we actually use for our loss function. This implements Uncertainty Weighting

In [11]:
class MultiTaskModelWrapper(nn.Module):
    def __init__(self):
        super(MultiTaskModelWrapper, self).__init__()
        self.log_vars = nn.Parameter(torch.zeros((len(tasks))))
        
    def forward(self, outputs, labels, criterion, sampling):
        mask = (torch.isnan(labels) == False)
        outputs = outputs * mask
        labels = torch.nan_to_num(labels, nan=0.0)
        if sampling == 'UAR':
            loss_treecover = criterion(outputs[:,0], labels[:, 0])
            precision_treecover = torch.exp(-self.log_vars[0])
            loss_treecover = precision_treecover * loss_treecover + self.log_vars[0]
            
            loss_elevation = criterion(outputs[:,1], labels[:, 1])
            precision_elevation = torch.exp(-self.log_vars[1])
            loss_elevation = precision_elevation * loss_elevation + self.log_vars[1]
            
            loss_population = criterion(outputs[:,2], labels[:, 2])
            precision_population = torch.exp(-self.log_vars[2])
            loss_population = precision_population * loss_population + self.log_vars[2]
            
            return loss_treecover + loss_elevation + loss_population
        elif sampling == "POP":
            loss_nightlights = criterion(outputs[:,0], labels[:, 0])
            precision_nightlights = torch.exp(-self.log_vars[3])
            loss_nightlights = precision_nightlights * loss_nightlights + self.log_vars[3]
            
            loss_income = criterion(outputs[:,1], labels[:, 1])
            precision_income = torch.exp(-self.log_vars[4])
            loss_income = precision_income * loss_income + self.log_vars[4]
            
            loss_roads = criterion(outputs[:,2], labels[:, 2])
            precision_roads = torch.exp(-self.log_vars[5])
            loss_roads = precision_roads * loss_roads + self.log_vars[5]
            
            loss_housing = criterion(outputs[:,3], labels[:, 3])
            precision_housing = torch.exp(-self.log_vars[6])
            loss_housing = precision_housing * loss_housing + self.log_vars[6]
            
            return loss_nightlights + loss_income + loss_roads + loss_housing

The two cells below just help test to see if the models look correct.

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = MultiTaskModel().to(device)
loss_ft = MultiTaskModelWrapper().to(device)

In [13]:
print(summary(model_ft, input_size=(32, 3, 224, 224), sampling="POP"))
print(summary(loss_ft))
for name, param in model_ft.named_parameters():
    if param.requires_grad:
        print(name)

Layer (type:depth-idx)                        Output Shape              Param #
MultiTaskModel                                --                        --
├─ModuleList: 1-1                             --                        --
├─ResNet: 1-2                                 [32, 512]                 --
│    └─Conv2d: 2-1                            [32, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                       [32, 64, 112, 112]        128
│    └─ReLU: 2-3                              [32, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [32, 64, 56, 56]          --
│    └─Sequential: 2-5                        [32, 64, 56, 56]          --
│    │    └─BasicBlock: 3-1                   [32, 64, 56, 56]          73,984
│    │    └─BasicBlock: 3-2                   [32, 64, 56, 56]          73,984
│    └─Sequential: 2-6                        [32, 128, 28, 28]         --
│    │    └─BasicBlock: 3-3                   [32, 128, 28, 28]         230,144
│  

In [15]:
def train_model(
    sampling,
    model_uuid,
    model,
    loss_model,
    criterion,
    train_dataloaders,
    test_dataloaders,
    optimizer,
    scheduler,
    mean,
    std,
    num_epochs=50,
    log_loc="./pytorch.logs",
    save_dir=Path(c.data_dir)/"int"/"deep_models",
    use_loss_model = False
):
    since = time.time()
    summary_writer = SummaryWriter(Path(log_loc)/ f"1234{sampling}")
    global_step = 0
    
    preds = {}
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    logger.debug("Using torch.device: {}".format(device))
    logger.debug("model uuid: {}".format(model_uuid))
    
    task_list = tasks_UAR if sampling == "UAR" else tasks_POP
    task_num = len(task_list)
    
    for epoch in range(num_epochs):
        logger.debug("Epoch {}/{}".format(epoch + 1, num_epochs))
        logger.debug("-" * 10)
        
        for phase in ["train", "test"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            all_labels = {sample : [] for sample in tasks}
            all_predictions = {sample : [] for sample in tasks}
            all_ids = {sample : [] for sample in tasks}

            counter = 0
            lr = optimizer.param_groups[0]["lr"]
            summary_writer.add_scalar(
                tag="learning_rate", scalar_value=lr, global_step=global_step
            )

            dataloaders = train_dataloaders if phase == "train" else test_dataloaders

            num_batches = len(dataloaders[sampling])
            logger.debug("Total batches: {}".format(num_batches))
            end_time = time.time()
            debug_time = time.time()
            
            loss_sum = 0
            
            for ids, inputs, labels in tqdm(dataloaders[sampling]):
                counter += 1
                global_step += 1
                    
                for i in range(task_num):
                    all_labels[task_list[i]] += list(labels.numpy()[:,i])
                    all_ids[task_list[i]] += list(ids)
                
                inputs = inputs.float()
                labels = labels.float()
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model.forward(inputs, sampling)

                    for i in range(task_num):
                        all_predictions[task_list[i]] += list(outputs.detach().cpu().numpy()[:,i])
                        
                    if not use_loss_model:
                        mask = (torch.isnan(labels) == False)
                        outputs = outputs * mask
                        labels = torch.nan_to_num(labels, nan=0.0)
                        loss = criterion(outputs, labels)
                        weight = F.softmax(torch.randn(task_num), dim=-1).to(device)
                        if sampling == "UAR":
                            reg_2 = torch.norm(model.sampling.UAR[3].weight, p=1)
                        else:
                            reg_2 = torch.norm(model.sampling.POP[3].weight, p=1)
                        final_loss = torch.sum(loss*weight) + reg_2 * 0.01
                        
                    else:
                        if sampling == "UAR":
                            reg = torch.norm(model.sampling.UAR.weight, p=1)
                        else:
                            reg = torch.norm(model.sampling.POP.weight, p=1)
                        loss = loss_model.forward(outputs, labels, criterion, "UAR")
                        final_loss = loss + reg * 0.1
                    
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                        summary_writer.add_scalar(
                            tag="train_loss",
                            scalar_value=loss.item(),
                            global_step=global_step,
                        )
                    else:
                        summary_writer.add_scalar(
                            tag="val_loss",
                            scalar_value=loss.item(),
                            global_step=global_step,
                        )
                loss_sum += loss.item()
                
            logger.debug("Time for {} batches: {}. Avg loss: {}".format(num_batches, time.time() - debug_time, loss_sum / num_batches))
            loss_sum = 0
            debug_time = time.time()

            # Testin some stuff here
            r2_scores = []
            for i in range(task_num):
                samp = task_list[i]
                temp_labels = np.array(all_labels[samp])
                temp_pred = np.array(all_predictions[samp])

                mask = (np.isnan(temp_labels) == False)
                temp_pred = temp_pred[mask]
                temp_labels = temp_labels[mask]

                temp_pred = temp_pred * std[i] + mean[i]
                temp_labels = temp_labels * std[i] + mean[i]

                r2_score = sklearn.metrics.r2_score(temp_labels, temp_pred)
                r2_scores.append(r2_score)
                logger.debug("Aggregate R2 for {0}: {1}".format(samp, r2_score))
            
            r2_scores = np.array(r2_scores)
            
            preds[phase] = (all_labels, all_predictions, all_ids)

            bio = python_io.BytesIO()
            torch.save(model.state_dict(), bio)
            model_checkpoint = {}
            model_checkpoint["model_bytes"] = bio.getvalue()
            model_checkpoint["val_r2"] = r2_scores
            model_checkpoint["epoch"] = epoch
            model_checkpoint["preds"] = preds[phase]
            model_checkpoint["domain_names"] = tasks_UAR

            if save_dir is not None:
                this_save_path = (
                    save_dir
                    / str(model_uuid)
                    / "checkpoints"
                    / phase
                    / f"epoch_{epoch}_UAR.pickle"
                )
                this_save_path.parent.mkdir(exist_ok=True, parents=True)
                with open(this_save_path, "wb") as f:
                    pickle.dump(model_checkpoint, f, protocol=4)
                
            if phase == "test":
                scheduler.step()

            logger.debug(
                "Epoch {0} Phase {1} complete".format(epoch + 1, phase)
            )
    
    time_elapsed = time.time() - since
    logger.debug(
        "Training complete in {:.0f}m {:.0f}s".format(
            time_elapsed // 60, time_elapsed % 60
        )
    )
    
    return model
            

In [16]:
def run(
    sampling,
    model_uuid,
    ids_train,
    Y_train,
    ids_test,
    Y_test,
    model,
    data_home,
    loss,
    num_epochs=50,
    initial_lr=0.001,
    log_loc="./pytorch.logs",
    save_dir=Path(c.data_dir)/"int"/"deep_models",
    batch_size=16,
):
    mean = np.nanmean(Y_train, axis=0)
    std = np.nanstd(Y_train, axis=0)
    Y_train = (Y_train - mean) / std
    Y_test = (Y_test - mean) / std
    
    train_dataloaders = {}
    test_dataloaders = {}
    
    train_dataloaders[sampling] = get_dataloader(
        data_home,
        Y_train,
        ids_train,
        batch_size=batch_size,
        augment=True)
    
    test_dataloaders[sampling] = get_dataloader(
        data_home,
        Y_test,
        ids_test,
        batch_size=batch_size)
    
    if loss == "mse":
        criterion = torch.nn.MSELoss()
    else:
        criterion = torch.nn.L1Loss()
        
    loss_model = MultiTaskModelWrapper()
        
    optimizer_ft = torch.optim.SGD(list(model.parameters()) + list(loss_model.parameters()), lr=initial_lr, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_ft, milestones=[10], gamma=0.5)
    
    if torch.cuda.is_available():
        model.cuda()
    
    return train_model(
        sampling,
        model_uuid,
        model,
        loss_model,
        criterion,
        train_dataloaders,
        test_dataloaders,
        optimizer_ft,
        scheduler,
        mean,
        std,
        num_epochs=num_epochs,
        log_loc=log_loc,
        save_dir=save_dir,
        use_loss_model=True
    )

In [17]:
model_uuid = uuid.uuid4()
trained_model = run(
    "UAR",
    model_uuid,
    ids_train_UAR,
    Y_train_UAR,
    ids_test_UAR,
    Y_test_UAR,
    MultiTaskModel(),
    data_home_UAR,
    'mse',
    num_epochs=50)

2022-06-01 01:22:15.280 | DEBUG    | __main__:train_model:24 - Using torch.device: cuda:0
2022-06-01 01:22:15.281 | DEBUG    | __main__:train_model:25 - model uuid: e990d040-0a61-4c10-ace2-8b4c4c4d38a7
2022-06-01 01:22:15.282 | DEBUG    | __main__:train_model:31 - Epoch 1/50
2022-06-01 01:22:15.283 | DEBUG    | __main__:train_model:32 - ----------
2022-06-01 01:22:15.285 | DEBUG    | __main__:train_model:53 - Total batches: 5000
  4%|████▌                                                                                                                | 196/5000 [00:41<16:51,  4.75it/s]


KeyboardInterrupt: 

In [19]:
# try save params
model_param_path = c.code_dir + "/cs230/temp_trained_params.pt"

# now try load
loaded_model = MultiTaskModel()
loaded_model.load_state_dict(torch.load(model_param_path))

RuntimeError: Error(s) in loading state_dict for MultiTaskModelTwo:
	Missing key(s) in state_dict: "sampling.UAR.2.weight", "sampling.UAR.2.bias", "sampling.POP.2.weight", "sampling.POP.2.bias". 
	Unexpected key(s) in state_dict: "sampling.UAR.3.weight", "sampling.UAR.3.bias", "sampling.POP.3.weight", "sampling.POP.3.bias". 

# Generate features for image dataset.

In [29]:
def resize_images(images):
    images_resized = []
    for im in images:
        im = im[:, :, :3]
        images_resized.append(
            skimage.transform.resize(
                im, (224, 224), mode="constant", anti_aliasing=True
            )
        )
    images = np.stack(images_resized, axis=0)
    return images

def resnet18multitask_features(images, model, batch_size=60, gpu=True):
    results = []
    if gpu:
        model = model.cuda()
    for images_chunk in chunks(images, batch_size):
        if len(images_chunk.shape) < 4:
            images_chunk = images[np.newaxis, :, :, :]
        images_chunk = images_chunk.astype("float32").transpose(0, 3, 1, 2)
        images_torch = torch.from_numpy(images_chunk)
        if gpu:
            images_torch = images_torch.cuda()
        x = model.resnet18(images_torch)
        x = x.view(x.size(0), -1)
        x = x.cpu().data.numpy()
        results.append(x)
    torch.cuda.empty_cache()
    return np.concatenate(results, axis=0)

def resnet18_features(images, model, batch_size=60, gpu=True):
    results = []
    if gpu:
        model = model.cuda()
    for images_chunk in chunks(images, batch_size):
        if len(images_chunk.shape) < 4:
            images_chunk = images[np.newaxis, :, :, :]
        images_chunk = images_chunk.astype("float32").transpose(0, 3, 1, 2)
        images_torch = torch.from_numpy(images_chunk)
        if gpu:
            images_torch = images_torch.cuda()
        x = model.conv1(images_torch)
        x = model.bn1(x)
        x = model.relu(x)
        x = model.maxpool(x)
        x = model.layer1(x)
        x = model.layer2(x)
        x = model.layer3(x)
        x = model.layer4(x)
        x = model.avgpool(x)
        x = x.view(x.size(0), -1)
        x = x.cpu().data.numpy()
        results.append(x)
    torch.cuda.empty_cache()
    return np.concatenate(results, axis=0)

def full_featurize(dataset, model_ft, model_type, batch_size, num_workers):
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )
    output_features = []
    data = {}
    ids = []
    print(len(dataloader))
    for j, what in tqdm(enumerate(dataloader)):
        i, X_batch = what[:2]
        t = time.time()
        X_batch = resize_images(X_batch.numpy())
        if model_type == 'resnet152':
            X_features = resnet152_features(X_batch, model_ft)
        elif model_type == 'resnet18multitask':
            X_features = resnet18multitask_features(X_batch, model_ft)
        else:
            X_features = resnet18_features(X_batch, model_ft)
        e = time.time()
        output_features.append(X_features)
    bio_features = python_io.BytesIO()
    np.save(bio_features, np.vstack(output_features))
    data["X"] = bio_features.getvalue()
    return data

In [30]:
model_path = "../../data/int/deep_models/d1c51538-7ca9-4cc6-8d6e-ea6e967ce077/checkpoints/test/epoch_45_UAR.pickle"
with open(model_path, "rb") as f:
    model_checkpoint = pickle.load(f)
print(model_checkpoint["val_r2"])
    
bio = python_io.BytesIO(model_checkpoint["model_bytes"])
multitaskmodel = MultiTaskModel()
multitaskmodel.load_state_dict(torch.load(bio))

[0.88263999 0.66210188 0.66960459]


<All keys matched successfully>

In [33]:
import importlib
import mosaiks.featurization
importlib.reload(mosaiks.featurization)
from mosaiks.featurization import RemoteSensingSubgridDataset
import dill

In [ ]:
model_type = "resnet18multitask"
for sample in ["UAR", "POP"]:

    if model_type == "resnet18multitask":
        model_ft = multitaskmodel
        out_file = c.features_dir + f"/{model_type}_{sample}_v2.pkl"
    else:
        model_ft = None
        out_file = c.features_dir + f"/{model_type}_combined_{sample}.pkl"

    subgrid_path = c.grid_paths[sample]

    subgrid = np.load(subgrid_path)
    ids = subgrid["ID"]
    latlons = np.hstack((subgrid["lat"][:, np.newaxis], subgrid["lon"][:, np.newaxis]))

    print(c.data_dir + f"/raw/imagery/CONTUS_{sample}")

    dataset = RemoteSensingSubgridDataset(c.data_dir + f"/raw/imagery/CONTUS_{sample}", latlons, ids)
    
    results_dict = full_featurize(dataset, model_ft, model_type, 128, 4)

    # This commented out section will generate features from each individual ResNet18
    # dataset = SelfStorageImageDataset(image_dir, ll_formatted)
#     print(len(dataset))
#     for task in tasks_UAR + tasks_POP:
#         print(f"Running for {task}")
#         resnet18_path = f"../../data/output/cnn_comparison/resnet18_{task}.pickle"
#         with open(resnet18_path, "rb") as f:
#             resnet18_results = pickle.load(f)
#         resnet18_model = resnet18_results["model"]
#         test_r2 = resnet18_results["test_r2"]
#         train_r2 = resnet18_results["train_r2"]
#         initial_lr = resnet18_results["initial_lr"]
#         print(test_r2)
#         print(train_r2)
#         print(initial_lr)
#         model_ft = resnet18_model
#         results_dict[f"X_{task}"] = full_featurize(dataset, model_ft, model_type, this_c["batch_size"], 4)
    results_dict["latlon"] = latlons
    results_dict["ids_X"] = ids
    with open(out_file, "wb") as f:
        dill.dump(results_dict, f, protocol=4)

In [35]:
with open(out_file, "wb") as f:
    dill.dump(results_dict, f, protocol=4)